In [1]:
import os, sys
import numpy as np
import matplotlib.pyplot as plt
import pickle
from copy import deepcopy
import CBcm

In [2]:
from scipy import stats

import iminuit

In [3]:
sys.path.append('/home/jp/projects/atm_nu/mceq_tysak/muon_fit/modules')

In [4]:
common_settings = pickle.load(open('/home/jp/projects/atm_nu/ddm_results/final_settings/common_settings_daemon.pkl'))

In [5]:
import mufit
reload(mufit)
import mufit_plots as mfp
reload(mfp)

MUFIT: Using quickflux2!
MUFIT: Using quickflux2!
MufitPlots does not use the mceq flux evaluator directly
MufitPlots does not use the mceq flux evaluator directly


<module 'mufit_plots' from '/home/jp/projects/atm_nu/mceq_tysak/muon_fit/modules/mufit_plots.pyc'>

In [6]:
fitname = 'daemon_prd_fixGSF'
filename = '/home/jp/projects/atm_nu/ddm_results/'+fitname+'.pkl'

old_fitname = 'daemon_prd_submission'
old_filename = '/home/jp/projects/atm_nu/ddm_results/'+old_fitname+'.pkl'

savefig = False

In [7]:
fit = mufit.MuFit(datasets = ['bessfull',
                              'l3c', 
                              'opera', 
                              'minos',
                              'cms',
                              'deis',
                              'mutronratio',
                             'deisratio'],

                 flux_splines = 'daemon-v4',)


/home/jp/projects/atm_nu/mceq_tysak/muon_fit/modules/resources/daemonsplines_20230323_2.pkl
Entering version with 5 entries
Excluded params removed from known_params.
No calibration file for daemon-v4
('Did not find an average flux', KeyError('average',))
Adjusting DEIS name
daemon-v4
('Using errors in mode: ', 'mixed')
MUTRON ratio: adding ALL errors in quadrature
DEIS ratio: adding ALL errors in quadrature


In [8]:
physics_params = common_settings['physics_params']

In [9]:
my_flux_parameters = {}
for onep in physics_params:
    if 'GSF' in onep:
        my_flux_parameters[onep] = [0., True, -4, 4, 0., 1]
    elif ('31G' in onep) or ('158G' in onep):
        my_flux_parameters[onep] = [0., False, -10, 10, 0., 1]
    elif ('20T' in onep) or ('2P' in onep):
        if 'pi+' in onep:
            my_flux_parameters[onep] = [-0.3, False, -5., 5.,  0.,  1.8]
        elif 'pi-' in onep:
            my_flux_parameters[onep] = [0,False, -33,  33,    0,  10]
        elif 'K+' in onep:
            my_flux_parameters[onep]= [ 0., False, -19,  19,  0,  10]
        elif 'K-' in onep:
            my_flux_parameters[onep]=[-20., False, -25,  30, 10.,  20.]
        elif 'p' in onep:
            my_flux_parameters[onep]=[12., False, -16, 16,  0., 6]
        elif 'n' in onep:
            my_flux_parameters[onep]=[0., False, -6, 6,   0,  2.5]
#for onep in physics_params:
#    print(onep, my_flux_parameters[onep])
    
mysyst = deepcopy(fit.systematics)
mysyst.update({# Fixing DEIS because errors are included by default
               'deis_mscat':  [0.0, True, -3, 3],
               'deis_bfield':  [0.0, True, -3, 3],
               'deis_norm':  [0.0, True, -3, 3],

               # L3c (all)
               'l3c_align': 0.0042, #np.random.uniform(-3,3),
               'l3c_bfield': -3.30, #np.random.uniform(-3,3),
               'l3c_eloss': -2.92, #np.random.uniform(-3,3),
               'l3c_overb': -0.5, #np.random.uniform(-3,3),
               'l3c_normHor': -1.41, #np.random.uniform(-3,3),
               'l3c_normVert': -1.591, #np.random.uniform(-3,3),
               'l3c_pres': 1.352, #np.random.uniform(-3,3),

               'mn_escale': -1.59, #np.random.uniform(-3,3),

               # Bess
               'bess_align': -1.72, #np.random.uniform(-3,3),
               'bess_finres': 0.2, #np.random.uniform(-3,3),
               })

In [10]:
# Check if there's a previous fit, start from there
if os.path.isfile(old_filename):
    old_result = pickle.load(open(old_filename,'rb'))
    print('Updating starting values')
    for onekey in my_flux_parameters:
        my_flux_parameters[onekey][0] = old_result['params'][onekey]['value']
    for onekey in mysyst:
        if type(mysyst[onekey]) == list:
            mysyst[onekey][0] = old_result['params'][onekey]['value']    
        else:
            mysyst[onekey] = old_result['params'][onekey]['value'] 


Updating starting values


In [11]:
nfits = 1
counter = 0
bestfit = 1E9
defprior = 60
for ifit in range(nfits):
    print 'Internal fit ', ifit
    
    # Start at best fit from previous fit
    if bestfit<1E9:
        for onekey in my_flux_parameters:
            my_flux_parameters[onekey][0] = fit.results['params'][onekey]['value']
        for onekey in mysyst:
            if type(mysyst[onekey]) == list:
                mysyst[onekey][0] = fit.results['params'][onekey]['value']    
            else:
                mysyst[onekey] = fit.results['params'][onekey]['value']  
            
    # Change some parameters slightly
    for i in range(4):
        randkey = my_flux_parameters.keys()[np.random.randint(len(my_flux_parameters))]
        my_flux_parameters[randkey][0] += np.random.normal()
    
    #print(my_flux_parameters)
    
    one_m, one_f = fit.run_fit(
        flux_parameters = my_flux_parameters,        
        exp_parameters = mysyst,
        fit_data = 'all',
        fit_mode = 'physics',
        use_priors = True,
        print_mode = False,
        dry_run = False,
        emin=5.,
        emax=1E9,
        fake_data = False,
        )
    
    this_fit = fit.results['fmin']['fval']
    print 'This fit', this_fit
    if this_fit < bestfit:
        print 'Updated results for fit', this_fit, bestfit
        bestfit = deepcopy(this_fit)
        results = deepcopy(fit.results)
        m = one_m
        f = one_f

Internal fit  0


/home/jp/projects/atm_nu/mceq_tysak/muon_fit/modules/mufit.py:315: RuntimeWarning: invalid value encountered in divide
  phys_priors_mean)**2/phys_priors_err**2)[prior_bool]


Successful fit!
This fit 528.697079052
Updated results for fit 528.697079052 1000000000.0


In [12]:
free_params = 0
for one_param in fit.fit_keys:
    if not fit.minuit_settings['fix_'+one_param]:
        free_params += 1
#print(free_params, len(fit.fit_keys))

In [13]:
all_keys = results['params'].keys()
dof = fit.results['bins']-free_params
print 'Chi2',fit.results['chi2'], '/', dof, 'dof (approximate)'
p_value = 1-stats.chi2.cdf(fit.results['chi2'], dof)
print('P-value', p_value)
syst_keys = []
print '\t'.join(['Param', 'value','error'])
for one_param in physics_params:
    all_keys.remove(one_param)
    print '\t'.join([one_param, 
                     "%0.2f" % results['params'][one_param]['value'],
                    "%0.2f" % results['params'][one_param]['error']])
all_keys.sort()
for one_param in all_keys:
    print '\t'.join([one_param, 
                     "%0.2f" % results['params'][one_param]['value'],
                    "%0.2f" % results['params'][one_param]['error']])



Chi2 430.1822617056888 / 431 dof (approximate)
('P-value', 0.5020573493257341)
Param	value	error
pi+_31G	-2.16	0.78
pi-_31G	-3.25	0.38
K+_31G	-1.90	0.76
K-_31G	-0.28	0.99
pi+_158G	0.10	0.09
pi-_158G	4.10	0.67
K+_158G	0.44	0.98
K-_158G	0.26	0.99
p_158G	0.27	0.95
n_158G	1.99	0.78
pi+_20T	-1.90	0.24
pi-_20T	-5.80	1.76
pi+_2P	-4.46	1.40
pi-_2P	-3.13	7.26
K+_2P	7.57	6.40
K-_2P	-14.24	13.07
p_2P	10.97	5.22
n_2P	3.24	2.25
GSF_1	0.83	0.50
GSF_2	1.50	0.50
GSF_3	0.13	0.50
GSF_4	0.66	0.50
GSF_5	-2.44	0.50
GSF_6	1.05	0.50
bess_align	-1.59	0.67
bess_finres	0.18	0.82
deis_bfield	0.00	0.50
deis_mscat	0.00	0.50
deis_norm	0.00	0.50
l3c_align	0.05	0.11
l3c_bfield	2.86	0.89
l3c_eloss	2.61	0.89
l3c_normHor	-2.26	0.46
l3c_normVert	-2.15	0.43
l3c_overb	-0.62	0.75
l3c_pres	0.97	0.40
mn_escale	-1.65	0.31


In [14]:
fit.results['flux_parameters'] = my_flux_parameters

In [15]:
pickle.dump(fit.results, open(filename, 'wb'))
print('Written', filename)

('Written', '/home/jp/projects/atm_nu/ddm_results/daemon_prd_fixGSF.pkl')


In [16]:
m.matrix(correlation=True)

----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|              |        K+_2P         p_2P     pi-_158G       pi+_2P        K-_2P       K-_31G       n_158G      pi+_31G      K+_158G      K-_158G      pi-_20T     pi+_158G       p_158G       K+_31G      pi+_20T      pi-_31G       pi-_2P         n_2P   l3c_bfield    l3c_overb    mn_escale   bess_align l3c_normVert  l3c_normHor    l3c_eloss  bess_finres     l3c_pres    l3c_align |
----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|        K+_2P |        1.000        0.025       -0.007       -0.196        0.878        0.001       -0.019       -0.008       -0.098        0.027       -0.417        0.106        0.009       -0.027       -0.607       -0.001       -0.057       -0.028       -0.061       -0.016        0.020        0.064        0.066        0.018       -0.036        0.000        0.387       -0.012 |
|         p_2P |        0.025        1.000       -0.164        0.036        0.060        0.005       -0.184        0.046        0.006       -0.002       -0.481       -0.284       -0.061        0.061       -0.609        0.136       -0.027       -0.135        0.061       -0.000       -0.006       -0.091       -0.086       -0.061        0.051        0.008       -0.041       -0.004 |
|     pi-_158G |       -0.007       -0.164        1.000        0.046       -0.010        0.026       -0.574        0.102        0.024       -0.091        0.117        0.653       -0.166        0.129        0.088        0.241        0.092       -0.068       -0.018        0.108        0.019        0.188        0.099        0.053       -0.022       -0.048       -0.080       -0.069 |
|       pi+_2P |       -0.196        0.036        0.046        1.000       -0.073       -0.002        0.039       -0.023        0.043       -0.017       -0.156        0.029        0.003       -0.032       -0.100       -0.073        0.194        0.054        0.059        0.035       -0.090       -0.027       -0.061       -0.051        0.046       -0.009        0.014       -0.003 |
|        K-_2P |        0.878        0.060       -0.010       -0.073        1.000       -0.001        0.050       -0.024        0.086       -0.049       -0.619       -0.102        0.000       -0.013       -0.555       -0.083       -0.311        0.082       -0.053       -0.021       -0.013        0.043        0.048        0.006       -0.030        0.004        0.381        0.010 |
|       K-_31G |        0.001        0.005        0.026       -0.002       -0.001        1.000       -0.028        0.013        0.000        0.001        0.001        0.015       -0.008        0.012       -0.003       -0.295       -0.001        0.004       -0.001       -0.000        0.001       -0.000        0.002        0.001       -0.001        0.000       -0.000       -0.001 |
|       n_158G |       -0.019       -0.184       -0.574        0.039        0.050       -0.028        1.000       -0.183       -0.015       -0.032       -0.031       -0.234       -0.232       -0.189        0.120       -0.628        0.014       -0.124        0.014        0.013       -0.011       -0.002       -0.016       -0.012        0.010       -0.005       -0.002        0.007 |
|      pi+_31G |       -0.008        0.046        0.102       -0.023       -0.024        0.013       -0.183        1.000       -0.015        0.002        0.026        0.0

In [17]:
fit.results['cov_params'] = m.list_of_vary_param()
ordered_matrix = np.zeros_like(fit.results['corr_matrix'])
# Need to reorder this business
syst_params = ['l3c_bfield','l3c_overb','l3c_normVert', 'bess_align','bess_finres',
               'l3c_normHor','l3c_eloss','l3c_pres','l3c_align', 'mn_escale']
syst_params.sort()
ordered_phys = np.zeros(len(physics_params))
ordered_syst = np.zeros(len(syst_params))
for ip, one_param in enumerate(physics_params):
    ordered_phys[ip] = fit.results['cov_params'].index(one_param)
for ip, one_param in enumerate(syst_params):
    ordered_syst[ip] = fit.results['cov_params'].index(one_param)
    
full_list = physics_params + syst_params
full_in = np.array(np.concatenate((ordered_phys, ordered_syst)),dtype=int)
for j, one_param in enumerate(full_list):
    for k in range(len(full_in)):
        ordered_matrix[j,k] = fit.results['corr_matrix'][full_in[j], full_in[k]]

ValueError: u'GSF_1' is not in list

In [ ]:
if not os.path.exists('/home/jp/projects/atm_nu/ddm_results/'+fitname):
    os.mkdir('/home/jp/projects/atm_nu/ddm_results/'+fitname)

In [ ]:
maxb = len(physics_params)
corr_matrix = deepcopy(ordered_matrix)

for i in range(corr_matrix.shape[0]):
    for j in range(corr_matrix.shape[0]):
        if i<j: corr_matrix[i,j]=0.

params = deepcopy(full_list)

# Adjust parameter names
for i, onep in enumerate(params):
    params[i] = common_settings['param_legend'][onep]

    
fig = plt.figure(figsize=(12*0.7,10*0.7))
ax = fig.add_subplot(111)
xmarks = np.arange(0.5, corr_matrix.shape[0], 1.)
plt.pcolor(corr_matrix[:maxb,:maxb], cmap='RdBu', vmin=-1, vmax=1)
plt.xticks(xmarks, params[:maxb], rotation=90)
plt.yticks(xmarks, params[:maxb])
plt.ylim(0,maxb)
plt.xlim(0,maxb)
ax.set_ylim(ax.get_ylim()[::-1])
ax.xaxis.tick_top()
plt.colorbar()
plt.tight_layout()


if savefig:
    print('Saving figures')
    fig.savefig('/home/jp/projects/atm_nu/ddm_results/'+fitname+'/corr_phys_noscale_'+fitname+'.png', dpi=300)
    fig.savefig('/home/jp/projects/atm_nu/ddm_results/'+fitname+'/corr_phys_noscale_'+fitname+'.pdf')


In [ ]:
maxb = None
corr_matrix = deepcopy(ordered_matrix)

for i in range(corr_matrix.shape[0]):
    for j in range(corr_matrix.shape[0]):
        if i<j: corr_matrix[i,j]=0.

params = deepcopy(full_list)

# Adjust parameter names
for i, onep in enumerate(params):
    params[i] = common_settings['param_legend'][onep]
    
fig = plt.figure(figsize=(14,10))
ax = fig.add_subplot(111)
xmarks = np.arange(0.5, corr_matrix.shape[0], 1.)
plt.pcolor(corr_matrix[:maxb,:maxb], cmap='RdBu', vmin=-1, vmax=1)
plt.xticks(xmarks, params[:maxb], rotation=90)
plt.yticks(xmarks, params[:maxb])
plt.ylim(0,maxb)
plt.xlim(0,maxb)
ax.set_ylim(ax.get_ylim()[::-1])
ax.xaxis.tick_top()
plt.colorbar()

if savefig:
    print('Saving figures')
    fig.savefig('/home/jp/projects/atm_nu/ddm_results/'+fitname+'/corr_full_'+fitname+'.png', dpi=300)
    fig.savefig('/home/jp/projects/atm_nu/ddm_results/'+fitname+'/corr_full_'+fitname+'.pdf')


In [ ]:
inb = 27
maxb = None
    
fig = plt.figure(figsize=(14,10))
ax = fig.add_subplot(111)
xmarks = np.arange(0.5, corr_matrix[inb:maxb,inb:maxb].shape[0], 1.)
plt.pcolor(corr_matrix[inb:maxb,inb:maxb], cmap='RdBu', vmin=-1, vmax=1)
plt.xticks(xmarks, params[inb:maxb], rotation=90)
plt.yticks(xmarks, params[inb:maxb])
plt.ylim(0,maxb)
plt.xlim(0,maxb)
ax.set_ylim(ax.get_ylim()[::-1])
ax.xaxis.tick_top()
plt.colorbar()
if savefig:
    print('Saving figures')
    fig.savefig('/home/jp/projects/atm_nu/ddm_results/'+fitname+'/corr_syst_'+fitname+'.png', dpi=300)
    fig.savefig('/home/jp/projects/atm_nu/ddm_results/'+fitname+'/corr_syst_'+fitname+'.pdf')


In [ ]:
# Plots of the stored fit fluxes, ratios directly - does not evaluate splines again
reload(mfp)
mfp.plot_comparison(fit, pullflag=10.,
                   emin=5)

In [ ]:
reload(mfp)

In [ ]:
# Plots of the stored fit fluxes, ratios directly - does not evaluate splines again
f1 = mfp.plotICRC21(fit, data_prefit = True)
if savefig:
    print('Saving figures')
    f1.savefig('/home/jp/projects/atm_nu/ddm_results/muon_results/flux_vert_'+fitname+'.pdf')

In [ ]:
reload(mfp)
f2 = mfp.plotICRC21(fit, data= 'ratio',data_prefit = True)
if savefig:
    print('Saving figures')
    f2.savefig('/home/jp/projects/atm_nu/ddm_results/muon_results/ratio_vert_'+fitname+'.pdf')

In [ ]:
# Plots of the stored fit fluxes, ratios directly - does not evaluate splines again
reload(mfp)
f3 = mfp.plotICRC21(fit, data='deis', data_prefit = True)
if savefig:
    print('Saving figures')
    f3.savefig('/home/jp/projects/atm_nu/ddm_results/muon_results/flux_deis_'+fitname+'.pdf')

In [ ]:
reload(mfp)
f1 = mfp.plotPulls(mufit=fit, pull_scale=20, log_pulls=False
                  )

In [ ]:
minos_errors = one_m.minos()
pickle.dump(fit.results, open(filename, 'wb'))
print('Written', filename)